In [ ]:
import spacy
import cupy as cp
nlp = spacy.load("nl_core_news_lg")

In [ ]:
def group_similar_sentences(text, similarity_threshold=0.75):
    doc = nlp(text)
    
    valid_sentences = []
    valid_embeddings = []

    # Extract embeddings for valid sentences
    for sent in doc.sents:
        embedding = cp.array(sent.vector)  # Convert to CuPy array
        if not cp.all(embedding == 0):  # Check if the embedding is not zero
            valid_sentences.append(sent.text.strip())
            valid_embeddings.append(embedding)

    # Convert valid embeddings to a CuPy array
    valid_embeddings = cp.array(valid_embeddings)

    if valid_embeddings.shape[0] == 0:
        return ["No valid sentences with embeddings found."]
    
    # Compute cosine similarities using CuPy without normalization
    similarities = cp.dot(valid_embeddings, valid_embeddings.T)

    # Ensure the diagonal is zero for comparison
    cp.fill_diagonal(similarities, 0)

    norms = cp.linalg.norm(valid_embeddings, axis=1)
    
    similarities = similarities / cp.outer(norms, norms)

    grouped_sentences = []
    current_group = [valid_sentences[0]]

    for i in range(1, len(valid_sentences)):
        similarity = similarities[0, i]  # Compare with the first sentence in the current group

        if similarity >= similarity_threshold:
            current_group.append(valid_sentences[i])
        else:
            grouped_sentences.append(current_group)
            current_group = [valid_sentences[i]]

    grouped_sentences.append(current_group)

    return grouped_sentences

In [ ]:
text = """Vanaf 28 september 2024 neemt Antwerpen de fakkel van het Ensorjaar over van Oostende met een veelzijdig en verrassend expoprogramma. Wat Antwerpen heeft met Ensor? Een gedeelde, verrassende blik voorbij het alledaagse. Die gaat al terug tot de tijd van Ensor zelf. Niet toevallig kwamen veel van zijn werken nog tijdens zijn leven in de Scheldestad terecht. Ze vormen vandaag de kern van de Ensor-collectie van het KMSKA en een vertrekpunt voor het Ensor Research Project. In zijn oeuvre laat Ensor zich - net als Antwerpen - kennen als een game-changer: vaak met een knipoog, soms dwars en altijd innovatief. Eigenschappen die Ensor tijdloos en relevant maken. Antwerpen kiest daarom voor verrassende invalshoeken om zijn werk te belichten. Hoe zien we echo’s van Ensor in de kunst, mode en fotografie? Hoe blijft hij inspireren en wat kunnen we vandaag nog van hem leren?"""

In [ ]:
grouped_sentences = group_similar_sentences(text)

In [ ]:
for i, group in enumerate(grouped_sentences, 1):
    print(f"Groep {i}:")
    for sentence in group:
        print(f" - {sentence}")